In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from scipy.spatial import cKDTree  # For spatial indexing
from scipy.stats import pearsonr  # For correlation calculation
from scipy.interpolate import make_interp_spline
import torch
import torch.nn as nn

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DQNAgent class definition (same as in your training code)
class DQNAgent(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNAgent, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, action_size)

        # Initialize weights
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)
        nn.init.xavier_uniform_(self.fc3.weight)
        nn.init.zeros_(self.fc3.bias)

        self.to(device)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return self.fc3(x)

# Sugarscape Environment for evaluation with RL
class SugarscapeEnvironmentEvaluator:
    def __init__(self, width, height, num_agents, seed=None, broadcast_radius=0):
        self.width = width
        self.height = height
        self.num_agents = num_agents
        self.seed = seed
        self.broadcast_radius = broadcast_radius

        # Set the initial seed for reproducibility
        if self.seed is not None:
            random.seed(self.seed)
            np.random.seed(self.seed)
            torch.manual_seed(self.seed)

        # Parameters for environment dynamics
        self.params = {
            'max_sugar': 5,
            'growth_rate': 1,
            'sugar_peak_frequency': 0.04,
            'sugar_peak_spread': 6,
            'job_center_duration': (40, 100),
            'vision_range': 1,  # Vision range
            'message_expiry': 30,  # Message expiry
            'max_messages': 5,  # Limit messages per agent
            'exploration_probability': 0.1
        }

        self.job_centers = []
        self.sugar = np.zeros((self.height, self.width), dtype=int)
        self.create_initial_sugar_peaks()
        self.agents = self.initialize_agents()
        self.agent_positions = set((agent['x'], agent['y']) for agent in self.agents)
        self.dead_agents = []

        self.population_history = []
        self.average_wealth_history = []
        self.gini_coefficient_history = []
        self.average_network_size_history = []  # For average network size per timestep
        self.timestep = 0

        # Load the trained DQN model
        vision = self.params['vision_range']
        max_messages = self.params['max_messages']
        state_size = 5 + (2 * vision + 1) ** 2 + (3 * max_messages)
        self.state_size = state_size
        self.action_size = 5  # Up, Down, Left, Right, Stay
        self.dqn_model = DQNAgent(self.state_size, self.action_size).to(device)
        self.dqn_model.load_state_dict(torch.load('dqn_q_network_final.pth', map_location=device))
        self.dqn_model.eval()

    # Create initial sugar peaks
    def create_initial_sugar_peaks(self, num_peaks=2):
        for _ in range(num_peaks):
            self.create_job_center()
        self.update_sugar_landscape()

    # Create a job center (sugar peak)
    def create_job_center(self):
        x, y = np.random.randint(0, self.width), np.random.randint(0, self.height)
        duration = np.random.randint(*self.params['job_center_duration'])
        self.job_centers.append({
            'x': x, 'y': y,
            'duration': duration,
            'max_sugar': self.params['max_sugar']
        })

    # Update sugar landscape based on job centers
    def update_sugar_landscape(self):
        self.sugar = np.zeros((self.height, self.width))
        for center in self.job_centers:
            x_grid, y_grid = np.meshgrid(np.arange(self.width), np.arange(self.height))
            distance = np.sqrt((x_grid - center['x']) ** 2 + (y_grid - center['y']) ** 2)
            sugar_level = center['max_sugar'] * np.exp(
                -distance ** 2 / (2 * self.params['sugar_peak_spread'] ** 2)
            )
            self.sugar += sugar_level
        self.sugar = np.clip(self.sugar, 0, self.params['max_sugar'])
        self.sugar = np.round(self.sugar).astype(int)

    # Initialize agents with unique positions
    def initialize_agents(self):
        agents = []
        available_positions = set(
            (x, y) for x in range(self.width) for y in range(self.height)
        )
        for i in range(self.num_agents):
            if not available_positions:
                break
            position = random.choice(tuple(available_positions))
            available_positions.remove(position)
            x, y = position
            agents.append(self.create_agent(i, x, y))
        return agents

    # Create an agent
    def create_agent(self, id, x, y):
        return {
            'id': id,
            'x': x,
            'y': y,
            'sugar': np.random.randint(40, 80),
            'metabolism': np.random.randint(1, 3),
            'vision': self.params['vision_range'],
            'broadcast_radius': self.broadcast_radius,
            'messages': deque(maxlen=self.params['max_messages']),
            'destination': None,
            'memory': deque(maxlen=10)  # Memory of past successful locations
        }

    # Get visible sugar within agent's vision
    def get_visible_sugar(self, agent):
        x, y = agent['x'], agent['y']
        vision = agent['vision']
        y_min = max(0, y - vision)
        y_max = min(self.height, y + vision + 1)
        x_min = max(0, x - vision)
        x_max = min(self.width, x + vision + 1)
        visible_area = self.sugar[y_min:y_max, x_min:x_max]
        return visible_area

    # Agents broadcast messages about sugar
    def broadcast_messages(self):
        if not self.agents:
            return  # No agents to broadcast

        positions = np.array([[agent['x'], agent['y']] for agent in self.agents])
        tree = cKDTree(positions)
        network_sizes = []

        # For each agent, find neighbors within broadcast radius
        for i, agent in enumerate(self.agents):
            # Identify visible sugar peaks
            visible_sugar = self.get_visible_sugar(agent)
            sugar_locations = np.argwhere(visible_sugar > 0)
            messages = []
            for loc in sugar_locations:
                msg_x = agent['x'] + loc[1] - agent['vision']
                msg_y = agent['y'] + loc[0] - agent['vision']
                # Ensure message coordinates are within grid
                msg_x = int(np.clip(msg_x, 0, self.width - 1))
                msg_y = int(np.clip(msg_y, 0, self.height - 1))
                msg = {
                    'sender_id': agent['id'],
                    'timestep': self.timestep,
                    'sugar_amount': self.sugar[msg_y, msg_x],
                    'x': msg_x,
                    'y': msg_y
                }
                messages.append(msg)

            # Broadcast to neighbors within broadcast_radius
            neighbors = tree.query_ball_point([agent['x'], agent['y']], agent['broadcast_radius'])
            network_size = len(neighbors) - 1  # Exclude the agent itself
            network_sizes.append(network_size)

            for neighbor_idx in neighbors:
                if neighbor_idx != i:
                    for msg in messages:
                        self.agents[neighbor_idx]['messages'].append(msg)

        # Calculate average network size for this timestep
        if network_sizes:
            average_network_size = sum(network_sizes) / len(network_sizes)
        else:
            average_network_size = 0
        self.average_network_size_history.append(average_network_size)

    # Move agent using the trained DQN model
    def move_agent(self, agent):
        x, y = agent['x'], agent['y']
        state = self.get_state(agent)
        valid_actions = self.get_valid_actions(agent)
        if not valid_actions:
            return  # No valid actions

        # Select action using the trained DQN model
        action = self.select_action(state, valid_actions)
        self.apply_action(agent, action)

    # Get the state representation for the agent
    def get_state(self, agent):
        x, y = agent['x'], agent['y']
        sugar = agent['sugar'] / 100  # Normalize sugar level
        metabolism = agent['metabolism'] / 5  # Normalize metabolism
        vision = agent['vision'] / 5  # Normalize vision

        # Extract sugar levels within vision range
        vision_range = agent['vision']
        y_min = max(0, y - vision_range)
        y_max = min(self.height, y + vision_range + 1)
        x_min = max(0, x - vision_range)
        x_max = min(self.width, x + vision_range + 1)
        sugar_map = self.sugar[y_min:y_max, x_min:x_max]

        # Pad the sugar map to a fixed size
        expected_size = (2 * vision_range + 1, 2 * vision_range + 1)
        pad_h = expected_size[0] - sugar_map.shape[0]
        pad_w = expected_size[1] - sugar_map.shape[1]

        pad_top = pad_h // 2
        pad_bottom = pad_h - pad_top
        pad_left = pad_w // 2
        pad_right = pad_w - pad_left

        padded_sugar_map = np.pad(
            sugar_map,
            ((pad_top, pad_bottom), (pad_left, pad_right)),
            mode='constant',
            constant_values=0
        )
        sugar_map_flat = padded_sugar_map.flatten() / self.params['max_sugar']

        # Encode messages
        N = self.params['max_messages']
        messages = list(agent['messages'])[-N:]
        message_features = []
        for msg in messages:
            # Normalize message coordinates relative to grid size
            msg_x = msg['x'] / self.width
            msg_y = msg['y'] / self.height
            msg_sugar = msg['sugar_amount'] / self.params['max_sugar']
            message_features.extend([msg_x, msg_y, msg_sugar])
        # Pad remaining messages with zeros if fewer than N
        while len(message_features) < 3 * N:
            message_features.extend([0.0, 0.0, 0.0])

        state = np.concatenate(
            (
                [x / self.width, y / self.height, sugar, metabolism, vision],
                sugar_map_flat,
                message_features
            )
        )
        return state

    # Get valid actions for the agent
    def get_valid_actions(self, agent):
        actions = []
        x, y = agent['x'], agent['y']
        possible_moves = {
            0: (x - 1, y),  # Left
            1: (x + 1, y),  # Right
            2: (x, y - 1),  # Up
            3: (x, y + 1),  # Down
            4: (x, y)       # Stay
        }
        for action, (nx, ny) in possible_moves.items():
            if 0 <= nx < self.width and 0 <= ny < self.height:
                if (nx, ny) not in self.agent_positions or (nx, ny) == (x, y):
                    actions.append(action)
        return actions

    # Select action using the trained DQN model
    def select_action(self, state, valid_actions):
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
        with torch.no_grad():
            q_values = self.dqn_model(state_tensor)
        q_values = q_values.cpu().numpy().flatten()
        # Mask invalid actions
        q_values_masked = np.full_like(q_values, -np.inf)
        q_values_masked[valid_actions] = q_values[valid_actions]
        action = np.argmax(q_values_masked)
        return action

    # Apply the selected action to the agent
    def apply_action(self, agent, action):
        x, y = agent['x'], agent['y']
        possible_moves = {
            0: (x - 1, y),  # Left
            1: (x + 1, y),  # Right
            2: (x, y - 1),  # Up
            3: (x, y + 1),  # Down
            4: (x, y)       # Stay
        }
        nx, ny = possible_moves[action]
        if (nx, ny) != (x, y):
            self.agent_positions.remove((x, y))
            agent['x'], agent['y'] = nx, ny
            self.agent_positions.add((nx, ny))

    # Collect sugar and update agent's memory
    def collect_sugar_and_update_memory(self, agent):
        collected_sugar = self.sugar[agent['y'], agent['x']]
        if collected_sugar > 0:
            agent['memory'].append((agent['x'], agent['y'], collected_sugar))
        agent['sugar'] += collected_sugar
        self.sugar[agent['y'], agent['x']] = 0
        agent['sugar'] -= agent['metabolism']
        agent['sugar'] = int(agent['sugar'])  # Ensure agent sugar is an integer

    # Perform one simulation step
    def step(self):
        # Update job centers and sugar landscape
        for center in self.job_centers:
            center['duration'] -= 1
        self.job_centers = [center for center in self.job_centers if center['duration'] > 0]
        if np.random.random() < self.params['sugar_peak_frequency']:
            self.create_job_center()
        self.update_sugar_landscape()

        # Move agents
        for agent in self.agents:
            self.move_agent(agent)

        # Collect sugar and apply metabolism
        for agent in self.agents:
            self.collect_sugar_and_update_memory(agent)

        # Broadcast messages and collect network size data
        self.broadcast_messages()

        # Clean up expired messages
        for agent in self.agents:
            agent['messages'] = deque(
                [msg for msg in agent['messages']
                 if self.timestep - msg['timestep'] <= self.params['message_expiry']],
                maxlen=self.params['max_messages']
            )

        # Handle agent death
        alive_agents = []
        for agent in self.agents:
            if agent['sugar'] <= 0:
                self.dead_agents.append({
                    'x': agent['x'],
                    'y': agent['y'],
                    'death_time': self.timestep
                })
                self.agent_positions.remove((agent['x'], agent['y']))
            else:
                alive_agents.append(agent)
        self.agents = alive_agents

        # Remove dead agents after 5 timesteps
        self.dead_agents = [
            agent for agent in self.dead_agents
            if self.timestep - agent['death_time'] <= 5
        ]

        self.collect_data()
        self.timestep += 1

    # Collect data for analysis
    def collect_data(self):
        population = len(self.agents)
        total_wealth = sum(agent['sugar'] for agent in self.agents)
        average_wealth = total_wealth / population if population > 0 else 0

        self.population_history.append(population)
        self.average_wealth_history.append(average_wealth)
        self.gini_coefficient_history.append(self.calculate_gini_coefficient())
        # Average network size is already collected in broadcast_messages()

    # Calculate Gini coefficient for wealth inequality
    def calculate_gini_coefficient(self):
        if not self.agents:
            return 0
        wealth_values = sorted(agent['sugar'] for agent in self.agents)
        cumulative_wealth = np.cumsum(wealth_values)
        n = len(wealth_values)
        total_wealth = cumulative_wealth[-1]
        if total_wealth == 0:
            return 0
        gini = (n + 1 - 2 * np.sum(cumulative_wealth) / total_wealth) / n
        return gini

# Function to run multiple simulations and collect averaged results for varying broadcast radii
def run_multiple_simulations_varying_broadcast_radius(broadcast_radii, num_runs=20, width=50, height=50, num_agents=1000,
                                                      max_timesteps=1000):
    # Store results for each broadcast radius
    results = []

    for broadcast_radius in broadcast_radii:
        print(f"\nRunning simulations for broadcast radius {broadcast_radius}")
        population_histories = []
        average_wealth_histories = []
        gini_coefficient_histories = []
        average_network_size_histories = []
        population_histories_full = []  # For plotting population vs timestep

        for run in range(num_runs):
            seed = run  # Different seed for each run
            env = SugarscapeEnvironmentEvaluator(width=width, height=height, num_agents=num_agents,
                                                 seed=seed, broadcast_radius=broadcast_radius)
            print(f"  Simulation {run + 1}/{num_runs} with seed {seed}...")
            for _ in range(max_timesteps):
                env.step()
            population_histories.append(env.population_history[-1])  # Final population
            average_wealth_histories.append(env.average_wealth_history[-1])  # Final average wealth
            gini_coefficient_histories.append(env.gini_coefficient_history[-1])  # Final Gini coefficient
            average_network_size_histories.append(np.mean(env.average_network_size_history))
            population_histories_full.append(env.population_history)  # Full population history

        # Convert lists to NumPy arrays for statistical analysis
        population_histories = np.array(population_histories)
        average_wealth_histories = np.array(average_wealth_histories)
        gini_coefficient_histories = np.array(gini_coefficient_histories)
        average_network_size_histories = np.array(average_network_size_histories)
        population_histories_full = np.array(population_histories_full)

        # Calculate average and standard deviation
        avg_population = np.mean(population_histories)
        std_population = np.std(population_histories)

        avg_wealth = np.mean(average_wealth_histories)
        std_wealth = np.std(average_wealth_histories)

        avg_gini = np.mean(gini_coefficient_histories)
        std_gini = np.std(gini_coefficient_histories)

        avg_network_size = np.mean(average_network_size_histories)
        std_network_size = np.std(average_network_size_histories)

        avg_population_history = np.mean(population_histories_full, axis=0)

        # Store the results
        result = {
            'broadcast_radius': broadcast_radius,
            'avg_population': avg_population,
            'std_population': std_population,
            'avg_wealth': avg_wealth,
            'std_wealth': std_wealth,
            'avg_gini': avg_gini,
            'std_gini': std_gini,
            'avg_network_size': avg_network_size,
            'std_network_size': std_network_size,
            'avg_population_history': avg_population_history  # For plotting population vs timestep
        }
        results.append(result)

    return results

# Function to plot results across different broadcast radii
def plot_results_across_broadcast_radii(results, max_timesteps=1000):
    """
    Plot the specified metrics across different broadcast radii using mean values and shaded areas for deviations.
    """
    # Create 'plots' directory if it doesn't exist
    os.makedirs('plots', exist_ok=True)

    # Extract broadcast radii and corresponding final metrics
    broadcast_radii = [res['broadcast_radius'] for res in results]
    final_avg_population = [res['avg_population'] for res in results]
    final_std_population = [res['std_population'] for res in results]

    final_avg_wealth = [res['avg_wealth'] for res in results]
    final_std_wealth = [res['std_wealth'] for res in results]

    final_avg_gini = [res['avg_gini'] for res in results]
    final_std_gini = [res['std_gini'] for res in results]

    avg_network_sizes = [res['avg_network_size'] for res in results]
    std_network_sizes = [res['std_network_size'] for res in results]

    # Create a figure with three subplots
    plt.figure(figsize=(18, 5))

    # Subplot 1: Final Population vs Broadcast Radius
    plt.subplot(131)
    plt.plot(broadcast_radii, final_avg_population, label='Average Population', color='blue', marker='o')
    plt.fill_between(broadcast_radii,
                     np.array(final_avg_population) - np.array(final_std_population),
                     np.array(final_avg_population) + np.array(final_std_population),
                     color='blue', alpha=0.2, label='Std Dev')
    plt.title('Final Population vs Broadcast Radius')
    plt.xlabel('Broadcast Radius')
    plt.ylabel('Final Average Population')
    plt.ylim(bottom=0)  # Ensure y-axis starts at 0
    plt.legend()

    # Save Subplot 1
    plt.savefig('plots/final_population_vs_broadcast_radius_rl.png')

    # Subplot 2: Final Average Wealth vs Broadcast Radius
    plt.subplot(132)
    plt.plot(broadcast_radii, final_avg_wealth, label='Average Wealth', color='orange', marker='o')
    plt.fill_between(broadcast_radii,
                     np.array(final_avg_wealth) - np.array(final_std_wealth),
                     np.array(final_avg_wealth) + np.array(final_std_wealth),
                     color='orange', alpha=0.2, label='Std Dev')
    plt.title('Final Average Wealth vs Broadcast Radius')
    plt.xlabel('Broadcast Radius')
    plt.ylabel('Final Average Wealth')
    plt.ylim(bottom=0)  # Ensure y-axis starts at 0
    plt.legend()

    # Save Subplot 2
    plt.savefig('plots/final_average_wealth_vs_broadcast_radius_rl.png')

    # Subplot 3: Final Gini Coefficient vs Broadcast Radius
    plt.subplot(133)
    plt.plot(broadcast_radii, final_avg_gini, label='Average Gini Coefficient', color='green', marker='o')
    plt.fill_between(broadcast_radii,
                     np.array(final_avg_gini) - np.array(final_std_gini),
                     np.array(final_avg_gini) + np.array(final_std_gini),
                     color='green', alpha=0.2, label='Std Dev')
    plt.title('Final Gini Coefficient vs Broadcast Radius')
    plt.xlabel('Broadcast Radius')
    plt.ylabel('Final Gini Coefficient')
    plt.legend()

    # Save Subplot 3
    plt.savefig('plots/final_gini_coefficient_vs_broadcast_radius_rl.png')

    plt.tight_layout()
    plt.show()

    # Additional plot showing correlation between broadcast radius and final population
    # Compute Pearson correlation coefficient
    corr_coef, p_value = pearsonr(broadcast_radii, final_avg_population)

    # Create a new figure for the correlation plot
    plt.figure(figsize=(8, 6))
    plt.scatter(broadcast_radii, final_avg_population, color='blue', label='Data Points')

    # Create a smooth line using spline interpolation if sufficient data points
    if len(broadcast_radii) >= 4:
        # Sort the data for proper spline fitting
        sorted_indices = np.argsort(broadcast_radii)
        sorted_radii = np.array(broadcast_radii)[sorted_indices]
        sorted_population = np.array(final_avg_population)[sorted_indices]

        # Perform spline interpolation
        spline = make_interp_spline(sorted_radii, sorted_population, k=3)  # cubic spline
        radii_smooth = np.linspace(sorted_radii.min(), sorted_radii.max(), 500)
        population_smooth = spline(radii_smooth)
        plt.plot(radii_smooth, population_smooth, "r--", label='Smooth Fit Line')
    else:
        # If too few points for spline, just connect them with lines
        plt.plot(broadcast_radii, final_avg_population, "r--", label='Line Connecting Points')

    plt.title(f'Correlation between Broadcast Radius and Final Population\n'
              f'Pearson r = {corr_coef:.2f}, p = {p_value:.2e}')
    plt.xlabel('Broadcast Radius')
    plt.ylabel('Final Average Population')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('plots/correlation_broadcast_radius_final_population_rl.png')
    plt.show()

    # Plotting Population vs Timestep for each Broadcast Radius
    plt.figure(figsize=(10, 6))
    colors = plt.cm.viridis(np.linspace(0, 1, len(results)))
    for idx, res in enumerate(results):
        plt.plot(res['avg_population_history'], label=f"Broadcast Radius {res['broadcast_radius']}",
                 color=colors[idx])
    plt.title('Population over Time for Different Broadcast Radii (RL Controlled)')
    plt.xlabel('Timestep')
    plt.ylabel('Average Population')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('plots/population_over_time_different_broadcast_radii_rl.png')
    plt.show()

    # Plotting Average Network Size vs Broadcast Radius
    plt.figure(figsize=(8, 6))
    plt.plot(broadcast_radii, avg_network_sizes, label='Average Network Size', color='purple', marker='o')
    plt.fill_between(broadcast_radii,
                     np.array(avg_network_sizes) - np.array(std_network_sizes),
                     np.array(avg_network_sizes) + np.array(std_network_sizes),
                     color='purple', alpha=0.2, label='Std Dev')
    plt.title('Average Network Size vs Broadcast Radius')
    plt.xlabel('Broadcast Radius')
    plt.ylabel('Average Network Size')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('plots/average_network_size_vs_broadcast_radius_rl.png')
    plt.show()

# Main execution
if __name__ == "__main__":
    # Define the set of broadcast radii to test
    broadcast_radii = [0, 5, 15, 30, 50, 100]

    # Run simulations for each broadcast radius
    results = run_multiple_simulations_varying_broadcast_radius(
        broadcast_radii=broadcast_radii,
        num_runs=20,  # Set to 20 as per your requirement
        width=50,
        height=50,
        num_agents=1000,
        max_timesteps=1000
    )

    # Plot the results
    plot_results_across_broadcast_radii(results)

    print("\nAll simulations completed and results saved.")



Running simulations for broadcast radius 0


RuntimeError: Error(s) in loading state_dict for DQNAgent:
	size mismatch for fc1.weight: copying a param with shape torch.Size([128, 29]) from checkpoint, the shape in current model is torch.Size([128, 180]).